#  Course on Introduction to Power Systems
Professor: David Pozo
Skoltech, Feb-Mar 2018

## Simple DC-OPF: 3-bus example

In [1]:
using JuMP
using GLPKMathProgInterface

In [2]:
###  Data definition
# Ybus
X = [0.1, 0.1, 0.1];

# Generator limits
Pmin = [0, 0, 0];
Pmax = [3, 0.8, 0];

# Line limits
PLmax = [0.25, 2, 2];

# Voltage limits
Vmin = 0.95;
Vmax = 1.10;

# Demand
Pd = [0 0 0.8];

# Generation costs
c = [2,1, 0];

# Incidence Matrix [I x L]
A =[1   1   0
    -1  0   1
    0   -1  -1];

In [3]:
## Model definition
m = Model(solver=GLPKSolverLP())

@variable(m, p[i=1:3])
@variable(m, pl[l=1:3])
@variable(m, v[i=1:3])
@variable(m, θ[i=1:3])


# Nodal balance equation
@constraint(m, BalanceEq[i=1:3], p[i] - Pd[i] == sum(A[i,l]*pl[l] for l=1:3));

# Branch power flow
@constraint(m, FlowDefi[l=1:3], pl[l] == (1/X[l])*(sum(A[i,l]*θ[i] for i=1:3)));

# Line limits
@constraint(m, FlowLimits[l=1:3],  -PLmax[l] <= pl[l] <= PLmax[l]);

# Technical generation limits
@constraint(m, GenLimits[i=1:3], Pmin[i] <= p[i] <= Pmax[i]);

# Voltage limits
@constraint(m, VoltageLimits[i=1:3], Vmin <= v[i] <= Vmax);

# Angle limits
@constraint(m,  θ[1] == 0);
@constraint(m, limitAngle[i=2:3], -pi/2 <= θ[i] <= pi/2);

## Objective function
@objective(m, Min, sum(c[i]*p[i] for i=1:3))


# Print the model
print(m)


Min 2 p[1] + p[2]
Subject to
 p[1] - pl[1] - pl[2] = 0
 p[2] + pl[1] - pl[3] = 0
 p[3] + pl[2] + pl[3] = 0.8
 pl[1] - 10 θ[1] + 10 θ[2] = 0
 pl[2] - 10 θ[1] + 10 θ[3] = 0
 pl[3] - 10 θ[2] + 10 θ[3] = 0
 -0.25 ≤ pl[1] ≤ 0.25
 -2 ≤ pl[2] ≤ 2
 -2 ≤ pl[3] ≤ 2
 0 ≤ p[1] ≤ 3
 0 ≤ p[2] ≤ 0.8
 p[3] = 0
 0.95 ≤ v[1] ≤ 1.1
 0.95 ≤ v[2] ≤ 1.1
 0.95 ≤ v[3] ≤ 1.1
 θ[1] = 0
 -1.5707963267948966 ≤ θ[2] ≤ 1.5707963267948966
 -1.5707963267948966 ≤ θ[3] ≤ 1.5707963267948966
 p[i] ∀ i ∈ {1,2,3}
 pl[l] ∀ l ∈ {1,2,3}
 v[i] ∀ i ∈ {1,2,3}
 θ[i] ∀ i ∈ {1,2,3}


In [4]:
## resolution
solve(m)

:Optimal

In [5]:
getvalue(p)

3-element Array{Float64,1}:
 0.025
 0.775
 0.0  

In [6]:
# Locational Marginal Prices (LMP) from dual variables of the Balance Equation
getdual(BalanceEq)

3-element Array{Float64,1}:
 2.0
 1.0
 1.5

In [7]:
getvalue(pl)

3-element Array{Float64,1}:
 -0.25 
  0.275
  0.525